# Problem Statement : 

# Fake News Classification with The Help Of Natural Language Processing Technique. 
Fake news detection is a hot topic in the field of natural language processing.
We consume news through several mediums throughout the day in our daily routine, but sometimes it becomes difficult to decide which one is fake and which one is authentic. Our job is to create a model which predicts whether a given news is real or fake.

# Required Libraries

In [21]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1. Data Gathering

In [22]:
df = pd.read_csv("Data.csv")
df.head()

,Label,Tanggal,Judul,Narasi,Source
0,0,17 February 2024,"Autisme Bukan Penyakit, Psikolog: Ada 3 Cara M...",Psikolog Vitriani Sumarlis menjelaskan bahwa a...,https://www.kompas.com/edu/read/2024/02/17/125...
1,0,4 April 2024,Guru Besar UGM Luncurkan Buku Menolak Dehumani...,Guru Besar Departemen Sosiologi UGM Prof. Dr. ...,https://ugm.ac.id/id/berita/guru-besar-ugm-lun...
2,0,23 April 2024,"Hari Buku Sedunia Diperingati Setiap 23 April,...",Hari buku sedunia atau World Book Day dipering...,https://uici.ac.id/hari-buku-sedunia-diperinga...
3,0,15 May 2024,Keindahan Indonesia dalam Buku Fotografi Karya...,"Bagi penggemar buku-buku berkualitas, tanggal ...",https://kemenparekraf.go.id/ragam-ekonomi-krea...
4,0,30 May 2024,Greysia Polii Luncurkan Buku Biografi,Menjadi peraih medali emas Olimpiade adalah im...,https://pbdjarum.org/berita/diluar-arena/20240...


# 2. Data Analysis

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Label    1000 non-null   int64 
 1   Tanggal  1000 non-null   object
 2   Judul    1000 non-null   object
 3   Narasi   1000 non-null   object
 4   Source   1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB


In [24]:
df['Label'].value_counts()

1    642
0    358
Name: Label, dtype: int64

In [25]:
df.shape

(1000, 5)

In [26]:
df.isna().sum()

Label      0
Tanggal    0
Judul      0
Narasi     0
Source     0
dtype: int64

In [27]:
df = df.dropna() #Handled Missing values by droping those rows

In [28]:
df.isna().sum()

Label      0
Tanggal    0
Judul      0
Narasi     0
Source     0
dtype: int64

In [29]:
df.shape

(1000, 5)

In [30]:
df.reset_index(inplace=True)
df.head()

,index,Label,Tanggal,Judul,Narasi,Source
0,0,0,17 February 2024,"Autisme Bukan Penyakit, Psikolog: Ada 3 Cara M...",Psikolog Vitriani Sumarlis menjelaskan bahwa a...,https://www.kompas.com/edu/read/2024/02/17/125...
1,1,0,4 April 2024,Guru Besar UGM Luncurkan Buku Menolak Dehumani...,Guru Besar Departemen Sosiologi UGM Prof. Dr. ...,https://ugm.ac.id/id/berita/guru-besar-ugm-lun...
2,2,0,23 April 2024,"Hari Buku Sedunia Diperingati Setiap 23 April,...",Hari buku sedunia atau World Book Day dipering...,https://uici.ac.id/hari-buku-sedunia-diperinga...
3,3,0,15 May 2024,Keindahan Indonesia dalam Buku Fotografi Karya...,"Bagi penggemar buku-buku berkualitas, tanggal ...",https://kemenparekraf.go.id/ragam-ekonomi-krea...
4,4,0,30 May 2024,Greysia Polii Luncurkan Buku Biografi,Menjadi peraih medali emas Olimpiade adalah im...,https://pbdjarum.org/berita/diluar-arena/20240...


In [31]:
df['Judul'][0]

'Autisme Bukan Penyakit, Psikolog: Ada 3 Cara Mendampingi'

In [32]:
df = df.drop(['Tanggal','Narasi'],axis = 1)
df.head()

,index,Label,Judul,Source
0,0,0,"Autisme Bukan Penyakit, Psikolog: Ada 3 Cara M...",https://www.kompas.com/edu/read/2024/02/17/125...
1,1,0,Guru Besar UGM Luncurkan Buku Menolak Dehumani...,https://ugm.ac.id/id/berita/guru-besar-ugm-lun...
2,2,0,"Hari Buku Sedunia Diperingati Setiap 23 April,...",https://uici.ac.id/hari-buku-sedunia-diperinga...
3,3,0,Keindahan Indonesia dalam Buku Fotografi Karya...,https://kemenparekraf.go.id/ragam-ekonomi-krea...
4,4,0,Greysia Polii Luncurkan Buku Biografi,https://pbdjarum.org/berita/diluar-arena/20240...


# 3. Data Preprocessing

## 1.Tokenization

In [33]:
sample_data = 'The quick brown fox jumps over the lazy dog'
sample_data = sample_data.split()
sample_data

['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

## 2. Make Lowercase

In [34]:
sample_data = [data.lower() for data in sample_data]
sample_data

['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

## 3. Remove Stopwords

In [35]:
stopwords = stopwords.words('indonesian')
print(stopwords[0:10])
print(len(stopwords))

['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir']
758


In [36]:
sample_data = [data for data in sample_data if data not in stopwords]
print(sample_data)
len(sample_data)

['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']


9

## 4. Stemming

In [37]:
ps = PorterStemmer()
sample_data_stemming = [ps.stem(data) for data in sample_data]
print(sample_data_stemming)

['the', 'quick', 'brown', 'fox', 'jump', 'over', 'the', 'lazi', 'dog']


## 5. Lemmatization

In [38]:
import nltk
nltk.download('wordnet')
lm = WordNetLemmatizer()
sample_data_lemma = [lm.lemmatize(data) for data in sample_data]
print(sample_data_lemma)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fikri\AppData\Roaming\nltk_data...


['the', 'quick', 'brown', 'fox', 'jump', 'over', 'the', 'lazy', 'dog']


In [39]:
lm = WordNetLemmatizer()
corpus = []
for i in range (len(df)):
    review = re.sub('^a-zA-Z0-9', ' ', df['Judul'][i])
    review = review.lower()
    review = review.split()
    review = [lm.lemmatize(x) for x in review if x not in stopwords]
    review = " ".join(review)
    corpus.append(review)

In [40]:
len(corpus)

1000

In [41]:
df['Judul'][0]

'Autisme Bukan Penyakit, Psikolog: Ada 3 Cara Mendampingi'

In [42]:
corpus[0]

'autisme penyakit, psikolog: 3 mendampingi'

# 4. Vectorization (Convert Text data into the Vector)

In [43]:
tf = TfidfVectorizer()
x = tf.fit_transform(corpus).toarray()

In [44]:
y = df['Label']
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Label, dtype: int64

## Data splitting into the train and test

In [45]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 10, stratify = y )

In [46]:
len(x_train),len(y_train)

(700, 700)

In [47]:
len(x_test), len(y_test)

(300, 300)

# 5. Model Building

In [48]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

RandomForestClassifier()

# 6. Model Evaluation

In [49]:
y_pred = rf.predict(x_test)
accuracy_score_ = accuracy_score(y_test,y_pred) 
accuracy_score_

0.8

In [50]:
class Evaluation:
    
    def __init__(self,model,x_train,x_test,y_train,y_test):
        self.model = model
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test
        
    def train_evaluation(self):
        y_pred_train = self.model.predict(self.x_train)
        
        acc_scr_train = accuracy_score(self.y_train,y_pred_train)
        print("Accuracy Score On Training Data Set :",acc_scr_train)
        print()
        
        con_mat_train = confusion_matrix(self.y_train,y_pred_train)
        print("Confusion Matrix On Training Data Set :\n",con_mat_train)
        print()
        
        class_rep_train = classification_report(self.y_train,y_pred_train)
        print("Classification Report On Training Data Set :\n",class_rep_train)
        
        
    def test_evaluation(self):
        y_pred_test = self.model.predict(self.x_test)
        
        acc_scr_test = accuracy_score(self.y_test,y_pred_test)
        print("Accuracy Score On Testing Data Set :",acc_scr_test)
        print()
        
        con_mat_test = confusion_matrix(self.y_test,y_pred_test)
        print("Confusion Matrix On Testing Data Set :\n",con_mat_test)
        print()
        
        class_rep_test = classification_report(self.y_test,y_pred_test)
        print("Classification Report On Testing Data Set :\n",class_rep_test)
        
        

In [51]:
#Checking the accuracy on training dataset

Evaluation(rf,x_train, x_test, y_train, y_test).train_evaluation()


Accuracy Score On Training Data Set : 1.0

Confusion Matrix On Training Data Set :
 [[251   0]
 [  0 449]]

Classification Report On Training Data Set :
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       251
           1       1.00      1.00      1.00       449

    accuracy                           1.00       700
   macro avg       1.00      1.00      1.00       700
weighted avg       1.00      1.00      1.00       700



In [52]:
#Checking the accuracy on testing dataset
Evaluation(rf,x_train, x_test, y_train, y_test).test_evaluation()

Accuracy Score On Testing Data Set : 0.8

Confusion Matrix On Testing Data Set :
 [[ 50  57]
 [  3 190]]

Classification Report On Testing Data Set :
               precision    recall  f1-score   support

           0       0.94      0.47      0.62       107
           1       0.77      0.98      0.86       193

    accuracy                           0.80       300
   macro avg       0.86      0.73      0.74       300
weighted avg       0.83      0.80      0.78       300



# Prediction Pipeline

In [53]:
class Preprocessing:
    
    def __init__(self,data):
        self.data = data
        
    def text_preprocessing_user(self):
        lm = WordNetLemmatizer()
        pred_data = [self.data]    
        preprocess_data = []
        for data in pred_data:
            review = re.sub('^a-zA-Z0-9',' ', data)
            review = review.lower()
            review = review.split()
            review = [lm.lemmatize(x) for x in review if x not in stopwords]
            review = " ".join(review)
            preprocess_data.append(review)
        return preprocess_data    

In [54]:
df['Judul'][0]

'Autisme Bukan Penyakit, Psikolog: Ada 3 Cara Mendampingi'

In [55]:
data = "Depo Plumpang Terbakar, Anggota DPR Minta Pertamina Pastikan Pasokan BBM Tak Terganggu"
Preprocessing(data).text_preprocessing_user()

['depo plumpang terbakar, anggota dpr pertamina pastikan pasokan bbm terganggu']

In [56]:
class Prediction:
    
    def __init__(self,pred_data, model):
        self.pred_data = pred_data
        self.model = model
        
    def prediction_model(self):
        preprocess_data = Preprocessing(self.pred_data).text_preprocessing_user()
        data = tf.transform(preprocess_data)
        prediction = self.model.predict(data)
        
        if prediction [0] == 1 :
            return "The News Is Fake"
        
        else:
            return "The News Is Real"
        

In [57]:
data = "The News Is Fake"
Prediction(data,rf).prediction_model()

'The News Is Fake'

In [58]:
df['Judul'][11]

'Paus Fransiskus ke Indonesia'

In [59]:
user_data = 'Paus Fransiskus ke Indonesia'
Prediction(user_data,rf).prediction_model()

'The News Is Real'